In [1]:
import pandas
from pandas import *
import numpy
from datetime import datetime



if False:
    df_train = pandas.read_csv("../input/train.csv")
    df_train = df_train.sample(50000)
    df_test = pandas.read_csv("../input/test.csv")
else:
    # Get the training data
    df_train = pandas.read_csv("data/train.csv.gz")
    # Only small sample for now
    #
    df_train = df_train.sample(200000)
    # Reading test data set
    df_test = pandas.read_csv("data/test.csv.gz")


print("Starting with {0}".format(df_train.count()))
display(df_train.sample(10))
df_train.describe()

Starting with date     50000
store    50000
item     50000
sales    50000
dtype: int64


,date,store,item,sales
517971,2016-04-28,4,29,89
586686,2014-06-25,2,33,104
71597,2014-01-19,10,4,11
593133,2017-02-18,5,33,50
279739,2013-12-28,4,16,22
546771,2015-03-09,10,30,46
276625,2015-06-19,2,16,45
473553,2014-09-12,10,26,58
291742,2016-11-09,10,16,31
508828,2016-04-15,9,28,119


,store,item,sales
count,50000.00000,50000.000000,50000.000000
mean,5.50262,25.598660,52.184300
std,2.86886,14.438555,28.900406
min,1.00000,1.000000,3.000000
25%,3.00000,13.000000,30.000000
50%,6.00000,26.000000,47.000000
75%,8.00000,38.000000,70.000000
max,10.00000,50.000000,205.000000


In [2]:
# Adding various information to the dataframe for later 
# DayOfWeek
# DayOfMonth
# Month
# Year
# CalendarWeek

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

# Add all variables that we would like to this model

# Extracting some variable from date
def date_info(date):
    datetime_object = datetime.strptime(date, "%Y-%m-%d")
    
    week_day = datetime_object.weekday()
    cw = datetime_object.isocalendar()[1]
    
    day = datetime_object.day
    month = datetime_object.month
    year = datetime_object.year
    
    days = (datetime_object - datetime(2010,1,1)).days
    
    return (week_day, cw, day, days, month, year)

def boolean_value(value):
    global current_value
    return True


current_value = 0
def extrapolate(df):
    df['week_day'], df['cw'], df['day'], df['days'], df['month'], df['year'] = zip(*df["date"].map(date_info))

    # Convert date as proper time as well
    df['time'] = df['date'].astype('datetime64[ns]')
    
    # Month should be a category as well
    df['week_day'] = df['week_day'].astype('int32')
    df['month'] = df['month'].astype('int32')

    # Set type of store and item to category
    df['item'] = df['item'].astype('object')
    df['store'] = df['store'].astype('object')
    
    for i in range(1, 11):
        df['s_{0}'.format(i)] = (df['store'] == i)
        
    for i in range(1, 51):
        df['i_{0}'.format(i)] = (df['item'] == i)
    
    # Categorized store and item
    #enc = OneHotEncoder()
    #enc.fit(df['store'])  
    #OneHotEncoder(categorical_features='all', dtype='numpy.float64', handle_unknown='error', n_values='auto', sparse=True)
    #print(enc.transform(df['store']).toarray())
    
    #store_le = preprocessing.LabelEncoder()
    #store_le.fit(df['store'])
    #print(list(store_le.classes_))
    
    #store_le.transform(df['store']) 
    return df

    

# Add those information to df_train and df_test
df_train = extrapolate(df_train)
df_test = extrapolate(df_test)

# Understand how scale change depending on dow
df_dow = df_train
display(df_train.sample(10))


,date,store,item,sales,week_day,cw,day,days,month,year,...,i_41,i_42,i_43,i_44,i_45,i_46,i_47,i_48,i_49,i_50
768000,2015-12-17,1,43,33,3,51,17,2176,12,2015,...,False,False,True,False,False,False,False,False,False,False
866872,2016-09-10,5,48,52,5,36,10,2444,9,2016,...,False,False,False,False,False,False,False,True,False,False
181047,2013-10-01,10,10,75,1,40,1,1369,10,2013,...,False,False,False,False,False,False,False,False,False,False
1850,2013-01-25,2,1,17,4,4,25,1120,1,2013,...,False,False,False,False,False,False,False,False,False,False
389621,2014-11-15,4,22,87,5,46,15,1779,11,2014,...,False,False,False,False,False,False,False,False,False,False
376643,2014-05-03,7,21,32,5,18,3,1583,5,2014,...,False,False,False,False,False,False,False,False,False,False
522405,2013-06-19,7,29,46,2,25,19,1265,6,2013,...,False,False,False,False,False,False,False,False,False,False
277281,2017-04-05,2,16,29,2,14,5,2651,4,2017,...,False,False,False,False,False,False,False,False,False,False
416421,2013-04-04,9,23,24,3,14,4,1189,4,2013,...,False,False,False,False,False,False,False,False,False,False
595692,2014-02-21,7,33,34,4,8,21,1512,2,2014,...,False,False,False,False,False,False,False,False,False,False


In [3]:
# Generate our training/validation datasets
from sklearn import model_selection

result_cols = ['sales']
# Removing input_cols = ['store', 'item',
input_cols = [
              'week_day',
              'days',
              'cw',
              'day', 
              'days','month', 'year']

for i in range(1, 11):
    input_cols.append('s_{0}'.format(i))

for i in range(1, 51):
    input_cols.append('i_{0}'.format(i))

def get_values(df, cols=[]):
    final_df = df
    
    print(final_df.columns.values)
    # Remove all columns that are not inside the list
    for column in final_df.columns.values:
        if column not in cols:
            final_df = final_df.drop(columns=[column])
    return final_df.values


X = get_values(df_train, input_cols)
Y = get_values(df_train, result_cols).ravel()
X_test = get_values(df_test, input_cols)

validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

# Create validation + training set

['date' 'store' 'item' 'sales' 'week_day' 'cw' 'day' 'days' 'month' 'year'
 'time' 's_1' 's_2' 's_3' 's_4' 's_5' 's_6' 's_7' 's_8' 's_9' 's_10' 'i_1'
 'i_2' 'i_3' 'i_4' 'i_5' 'i_6' 'i_7' 'i_8' 'i_9' 'i_10' 'i_11' 'i_12'
 'i_13' 'i_14' 'i_15' 'i_16' 'i_17' 'i_18' 'i_19' 'i_20' 'i_21' 'i_22'
 'i_23' 'i_24' 'i_25' 'i_26' 'i_27' 'i_28' 'i_29' 'i_30' 'i_31' 'i_32'
 'i_33' 'i_34' 'i_35' 'i_36' 'i_37' 'i_38' 'i_39' 'i_40' 'i_41' 'i_42'
 'i_43' 'i_44' 'i_45' 'i_46' 'i_47' 'i_48' 'i_49' 'i_50']
['date' 'store' 'item' 'sales' 'week_day' 'cw' 'day' 'days' 'month' 'year'
 'time' 's_1' 's_2' 's_3' 's_4' 's_5' 's_6' 's_7' 's_8' 's_9' 's_10' 'i_1'
 'i_2' 'i_3' 'i_4' 'i_5' 'i_6' 'i_7' 'i_8' 'i_9' 'i_10' 'i_11' 'i_12'
 'i_13' 'i_14' 'i_15' 'i_16' 'i_17' 'i_18' 'i_19' 'i_20' 'i_21' 'i_22'
 'i_23' 'i_24' 'i_25' 'i_26' 'i_27' 'i_28' 'i_29' 'i_30' 'i_31' 'i_32'
 'i_33' 'i_34' 'i_35' 'i_36' 'i_37' 'i_38' 'i_39' 'i_40' 'i_41' 'i_42'
 'i_43' 'i_44' 'i_45' 'i_46' 'i_47' 'i_48' 'i_49' 'i_50']
['id' 'date' 'stor

In [4]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(X_train)  
OneHotEncoder(categorical_features='all', handle_unknown='error', n_values='auto', sparse=True)

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [ ]:
# Import algorithm
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeRegressor



# Test options and evaluation metric
seed = 7
scoring = 'accuracy'

# Spot Check Algorithms
models = []
#models.append(('LDA', LinearDiscriminantAnalysis()))
#models.append(('KNN', KNeighborsClassifier()))
#models.append(('CART', DecisionTreeClassifier()))
#models.append(('TREEREG', DecisionTreeRegressor()))

#models.append(('NB', GaussianNB()))
models.append(('NEURAL', MLPClassifier()))
#models.append(('LR', LogisticRegression()))
#models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []

import sklearn
print(sklearn.__version__)


# Simple function to represent the result
def mean_absolute_percentage_error(Y_validation, Y_validation_predict):
    result = 0
    for i in range(0, len(Y_validation)):
        result += (abs(Y_validation[i] - Y_validation_predict[i]))/(abs(Y_validation[i])+abs(Y_validation_predict[i]))
    return result / len(Y_validation) * 200
print("Starting all models")
#Y_train=Y_train.astype('int')

max_score = 10000
for name, model in models:
    print("Executing for model {0}".format(name))
    #kfold = model_selection.KFold(n_splits=10, random_state=seed)
    #cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    #results.append(cv_results)
    #names.append(name)
    #msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    #print(msg)
    
    # Writing ht
    
    # Training the model
    model.fit(X_train, Y_train)
    
    Y_validation_predict = model.predict(X_validation)
    score = mean_absolute_percentage_error(Y_validation, Y_validation_predict)
    print("Model {0} got score of {1}".format(name, score))
    
    Y_test = model.predict(X_test)
    print("Result should be {0}".format(Y_test))

    # Create new result dataframe:
    result_df = df_test['id']
    df_test['sales'] = Y_test
    df_test.head(5)
    df_result = df_test[['id', 'sales']]

    # Write the dataframe as output
    df_result.head(5)

    df_result.to_csv("output_{0}.csv".format(name), index=False)
    if max_score > score:
        max_score = score
        print("Choosing {0} as best result".format(name))
        df_result.to_csv("output_best.csv".format(name), index=False)
print("Final result should be around {0}".format(max_score))


0.19.2
Starting all models
Executing for model NEURAL
